# Importing Needed Libraries:

In [ ]:
import numpy as np
import cv2
import os
import threading
from IPython.display import Audio
import pyttsx3
import speech_recognition as sr
from playsound import playsound
import requests
import io
import json
import time

# Setting the environment:

In [ ]:
# enable working on gpu
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

# YoloV7 With AudioFeedBack:

In [ ]:
# function to return the detected_object_text as AudioFeedBack
def playing_audio():
    global out_text
    global current_text
    if (len(out_text)>0) : #and(current_text!=out_text)
        last_text=out_text 
        engine =pyttsx3.init()
        engine.say(out_text)
        engine.runAndWait()
        if engine._inLoop:
            engine.endLoop()
         

#### Note: the object detection Part (the part which run the yolov7) was written from https://github.com/HimanchalChandra/Object-Detection-with-Voice-Feedback-YOLO-v3-and-gTTS

In [ ]:
#function that detected the objects in the frame,
#write bounding boxes and labels on the frame
#then return the modified frame.
#it's also call the thread of the playing_audio function to 
#output the detected_object_text as AudioFeedBack
def video_stream_with_objectDetection(frame,frame_count):
    global text
    global texts
    global audio_text
    global Labels
    global COLORS
    global network
    global layer_names
    global current_text
    global out_text
    global detected_objects 

    key = cv2.waitKey(1)

    if (frame_count%100==0):
        if(current_text in detected_objects):
            detected_objects=[]
            if(current_text==out_text):
                detected_objects.append(current_text)  

    #get the frame dimension
    (H,W) =frame.shape[:2]

    # convert the frame to blob to give it to the network
    blob =cv2.dnn.blobFromImage(frame, 1/255.0, (416,416), crop=False, swapRB=True)

    network.setInput(blob)
    output_layers=network.forward(layer_names)

    # getting the output from the network

    boxes =[]
    confidences=[]
    classIDs=[]
    centers=[]

    for output in output_layers:
        for detection in output:
            #take the oneHotEncode vector that represent the label scores
            scores =detection[5:]
            # taking the index of the label with the highest score
            classID =np.argmax(scores)
            #get the highest confidence 
            confidence =scores[classID]

            #only take into consideration confidence higher than 0.6
            if (confidence >= 0.5):
                #getting x,y(bounding box centers) and w,h (width and high)
                box=detection[0:4]*np.array([W,H,W,H])
                (CenterX,CenterY,w,h) =box.astype("int")

                #getting the most bottom left of the box
                x=int(CenterX-(w/2))
                y=int(CenterY-(h/2))

                #append all the results
                boxes.append([x,y,int(w),int(h)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                centers.append((CenterY,CenterY))

        # removal of overlapping boxes
        idxs =cv2.dnn.NMSBoxes(boxes, confidences,0.5,0.3)

        #loop ovar detecion
    if (len(idxs) >0):
        #loop over deteciton
        for i in idxs.flatten():
            #extract x,y(bottom left point) and w,h (width and high)
            (x,y) = (boxes[i][0], boxes[i][1])
            (w,h) = (boxes[i][2], boxes[i][2])

            # draw the boxes and label on the frame
            color =[int(c) for c in COLORS[classIDs[i]]]
            
            # draw the box
            cv2.rectangle(frame, (x,y), (x+w,y+h),color,2)
            # write the label
            text ="{}: {:.4f}".format(Labels[classIDs[i]], confidences[i])
            cv2.putText(frame, text, (x,y-5),
            cv2.FONT_HERSHEY_SIMPLEX,1,color,2)

            detected =[]

            if frame_count%2 ==0:
                current_label =Labels[classIDs[i]]       
                #if(current_label not in detected): 
                    #find x,y centers of the current detected box
                x,y =centers[i][0],centers[i][1]
                text=""           
                if (x<=(W/3)):
                    text ="left "
                elif(x<=((W/3)*2)):
                    text ="center "
                else:
                    text ="right "

                if (y<=(H/3)):
                    text ="top "+text
                elif(y<=((H/3)*2)):
                    text ="mid "+text
                else:
                    text ="bottom "+text

                out_text =text+" "+ Labels[classIDs[i]]
                detected.append(current_label)
                
                #print("out_text: ",out_text)

                if(out_text not in detected_objects):
                    t2=threading.Thread(target=playing_audio)
                    t2.start()
                    detected_objects.append(out_text)

                current_text=out_text

    return frame     

In [ ]:
# function that take the frames from the camera and pass it
# to the video_stream_with_objectDetection function
# then display the modified frame in the output("webcam") window 
def playing_video(cap):
    global frame_count
    ret, frame = cap.read()
    #frame=cv2.flip(frame,1)
    frame_count += 1
    if ret:
        cv2.imshow("Webcam",video_stream_with_objectDetection(frame,frame_count)) 
     

# OCR with Simple Chatbot:

In [ ]:
# funciton to listen to the voice of the user through microphone
# and convert it to a text
# then return this text 
def listen(recognizer, microphone):
    print("listing")
    try:
        with microphone as source:
            recognizer.adjust_for_ambient_noise(source)
            recognizer.dynamic_energy_thereshold=1000
            audio=recognizer.listen(source, timeout=2)
            response=recognizer.recognize_google(audio)
            print(response)
            return response

    except sr.WaitTimeoutError:
        print("timeError")
        return ""

    except sr.UnknownValueError:
        print("couldn't recognize")
        return ""

    except sr.RequestError:
        print("Network error")
        return ""

In [ ]:
# function that ask the user if he want to read from existing file
# if yes: it will ask the user for the file name
# then will check if the file exist
# if yes: it will read the file
# if no: it will say that the file doesn't exist, and ask the user again 
# for the correct file name
def read_ocr_file():
    isExist=False
    file_name =""
    text=""
    recognizer =sr.Recognizer()
    microphone =sr.Microphone()
    engine=pyttsx3.init()
    engine.say("Do You Want me to read from an existing file")
    engine.runAndWait()
    while text=="":
        text =listen(recognizer, microphone)

    if text=="yes":    
        while not isExist:
            engine=pyttsx3.init()
            engine.say("Please, say the name of the file")
            engine.runAndWait()
            engine =None
            while file_name=="":
                file_name =listen(recognizer, microphone)
            
            isExist =os.path.exists(file_name)
            if(isExist):
                engine =pyttsx3.init()
                with open(file_name) as f:
                    for line in f:
                        text +=line+" "
                    #text_in_file =f.readlines()
                engine.say(text)
                engine.runAndWait()
                engine =None

                return True
            else:
                current_text="There is no file exist With the name"+file_name
                engine=pyttsx3.init()
                engine.say(current_text)
                engine.runAndWait()
                engine =None
                file_name=""
    elif(text=="no"):
        return False
    else:
        return False    

In [ ]:
# function to ask the user if he want to save the ocr_text to a file
# if yes: then it will ask the user for the name of the file
# then will save the current text to a file with the name taken
# if other: it will not do anything
def Save_ocr_file(ocr_output_text):    
    file_name_verify=""
    file_name=""
    text=""

    recognizer =sr.Recognizer()
    microphone =sr.Microphone()

    engine=pyttsx3.init()
    engine.say("Do You Want to save the current text To A file?")
    engine.runAndWait()
    engine =None

    while text =="":
        text =listen(recognizer, microphone)

    if text =="yes":

        while (file_name_verify !="yes"):
            file_name_verify=""
            file_name=""
            engine=pyttsx3.init()
            engine.say("Please Say the name of the file")
            engine.runAndWait()
            engine =None

            while file_name =="":
                file_name =listen(recognizer, microphone)  

            current_text="is"+file_name+"the name of the new file?"        
            engine=pyttsx3.init()
            engine.say(current_text)
            engine.runAndWait()
            engine =None

            while file_name_verify =="":
                file_name_verify =listen(recognizer, microphone)
 
        engine=pyttsx3.init()
        engine.say("Saving the new file, Please Wait")
        engine.runAndWait()
        engine =None

        # writing ocr_text_output to the new file
        with open(file_name, "w") as f:
            f.write(ocr_output_text)


        engine=pyttsx3.init()
        engine.say("file is saved")
        engine.runAndWait()   
        engine =None     


In [ ]:
# function that use Ocr.Space to get the image and send it the api
# throught an apiKey, then it take the responds and get the result text
# from it
# if the text is not empty: it will read it
# then will call the Save_ocr_file function to check if the user want to 
# save the file or not
# if the text is empty: it will tell the user that it couldn't scan the text
def edited_ocr(img): 
    print("ocr_funciton")
    url_api="https://api.ocr.space/parse/image"

    img_gray =cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    cv2.imwrite(filename="ocr_image.jpg",img=img_gray)
    file_img =cv2.imread("ocr_image.jpg")
    _, compressedImage=cv2.imencode(".jpg",file_img,[1,90])
    file_bytes=io.BytesIO(compressedImage)

    result = requests.post(url_api,
              files = {"ocr_image.jpg": file_bytes},
              data = {"apikey": "K81364605688957",
                      "language": "eng"})
    
    result =result.content.decode()
    result=json.loads(result)

    text =result.get("ParsedResults")[0].get("ParsedText")
    print(text)
    if (text != ""):
        engine=pyttsx3.init()
        engine.say(text)
        engine.runAndWait()
        engine =None
        Save_ocr_file(text)
    else:
        engine=pyttsx3.init()
        engine.say("couldn't scan the text, pleas hold it the text in a clear way")
        engine.runAndWait()
        engine =None

In [ ]:
# function to display the ocr_window untill the user say the word (start)
# then it will capture the image
def display_ocr_window(cap):
    global read_text
    global ocr_start
    global read_save_file
    global read_saved_file_for_the_firstTime
    ret, frame = cap.read()
    #frame=cv2.flip(frame,1)
    if ret:
        cv2.imshow("OCR_Detection",frame)
        cv2.waitKey(1)

        if(read_text)and(not read_save_file)and(ocr_start):       
            edited_ocr(frame)
            read_saved_file_for_the_firstTime=True
            read_text=False 
            read_save_file=True   

# Functions Setting: 

In [ ]:
# function to enable each function when the user say a specific word
# 1-> start object_detection
# 2-> start ocr window
# start -> scan the text
# exit -> exit the program
# the funciton uses the listen function to listen to the user's voice
# througth the microphone
def set_function():  
    global ocr_start
    global read_text
    global objectDetection_start
    global exit_Program
    global start_new_thread

    start_new_thread=False
    
    recognizer =sr.Recognizer()
    microphone =sr.Microphone()     
    text =listen(recognizer, microphone)
        
    if(text =="1"):  #"start object detection" 
        if(ocr_start):
            ocr_start=False
        objectDetection_start=True    
        
    elif(text =="scan"): #"start scanning the text"
        read_text=True

    elif(text =="2"): #"start ocr"
        if(objectDetection_start):
            objectDetection_start=False     
        ocr_start=True
        read_saved_file_for_the_firstTime=True
     
     
    elif(text =="exit"):
        exit_Program=True

    else:
        print("else: ",text)
 
    start_new_thread=True   

# Main Function:

In [ ]:
# this is the main function where all other functions are organized in it
text=""
audio_text=""
texts=[]
current_text=""
out_text=""
detected_objects =[]

exit_Program=False
objectDetection_start=False
ocr_start=False
read_text=False
start_new_thread=True

read_saved_file_for_the_firstTime =True

#"E:\\AlaaFiles\\yolov7\\coco.yaml"
#"E:\\AlaaFiles\\Amit_Gradution_Porject\\yolo.cfg", "E:\\AlaaFiles\\yolov7\\yolov7-tiny.weights


Labels =open("coco.yaml").read().strip().split('\n')

# initilalize a list of colors
np.random.seed(42)
COLORS =np.random.randint(0,255,size=(len(Labels),3),dtype="uint8")

# Weights_Path ="yolov7-tiny.weights" 
# Config_Path ="yolo.cfg"

Weights_Path ="yolov7-tiny.weights"  
Config_Path ="yolov7.cfg"

# load nueral network from darknet #readNetFromDarknet
network = cv2.dnn.readNetFromDarknet(Config_Path, Weights_Path)

# get layer names
layer_names =network.getLayerNames()

# get unconnected layers(output_layer) names
layer_names =[layer_names[i-1] for i in network.getUnconnectedOutLayers()]
 

filewrite =open("ocr_text","w")

frame_count = 0
first = True
frames = []
started=True


if __name__ =="__main__":
    engine =pyttsx3.init()
    engine.say("Amit Graduation Project")
    engine.say("Alaa Mohamed Sayed Ibrahim")
    engine.say("For Object Detection Please Say one")
    engine.say("For reading Text Please Say Two")
    engine.say("to exit the program Please Say Exit")
    engine.runAndWait()
    engine =None
    start=False
    set_function()
    
    cap = cv2.VideoCapture(0)
    # cap.set(3,416)
    # cap.set(4,416) 
    
    now =time.time()

    while(not exit_Program) :
        if((start_new_thread)):  
            t1=threading.Thread(target=set_function)
            t1.start()    

        if(objectDetection_start):
            playing_video(cap)

        elif(ocr_start):
            if(read_saved_file_for_the_firstTime):
                read_save_file =read_ocr_file()
                read_saved_file_for_the_firstTime=False
                print("in read function")

                
                #if( read_save_file): #not
                engine=pyttsx3.init()
                engine.say("When You are ready, Please Say scan")
                engine.runAndWait()
                engine =None

            display_ocr_window(cap)

    cap.release()
    cv2.destroyAllWindows()
 
    print("Done")